In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random

In [30]:
%cd /home/gautam/Downloads
df = pd.read_csv('Assignment3_train_data.csv')
X = df.loc[1:,'feature0':'feature783'].to_numpy()/255
Y_temp = df['label'].as_matrix()
Y = np.zeros([len(Y_temp),4])
for i in range(len(Y_temp)):
    temp = [0,0,0,0]
    temp[Y_temp[i]-1] = 1
    Y[i] = temp

/home/gautam/Downloads


/usr/lib/python3/dist-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [31]:
class Neural_Network():
    def __init__(self, neurons, hidden_acti, output_acti): # arguments: an array "neurons" consist of number of neurons for each layer, activation function to be used in hidden layers and activation function to be used in output layer
        self.inputSize = neurons[0] # Number of neurons in input layer
        self.outputSize = neurons[-1] # Number of neurons in output layer
        self.layers = len(neurons)
        self.w = []
        for i in range(len(neurons)-1): 
            self.w.append(np.random.uniform(1e-6,1e-2,(neurons[i],neurons[i+1]))) #weight matrix between layer i and layer i+1
        self.activationHidden = None # Activation funtion to be used in hidden layers
        self.activationOutput = None # Activation funtion to be used in output layer
        self.activationHiddenPrime = None # Derivative of the activation funtion to be used in hidden layers
        self.activationOutputPrime = None # Derivative of the activation funtion to be used in output layer
        
        if(hidden_acti == "sigmoid"):
            self.activationHidden = self.sigmoid
            self.activationHiddenPrime = self.sigmoidPrime
        else:
            self.activationHidden = self.linear
            self.activationHiddenPrime = self.linearPrime
            
        if(output_acti == "sigmoid"):
            self.activationOutput = self.sigmoid
            self.activationOutputPrime = self.sigmoidPrime
        else:
            self.activationOutput = self.linear
            self.activationOutputPrime = self.linearPrime
            
    def sigmoid(self, s): # sigmoid activation function
        return(1/(1+np.exp(-s)))
    
    def sigmoidPrime(self,x): # derivative sigmoid activation function
        return(self.sigmoid(x)*(1-self.sigmoid(x)))
    
    def linear(self, s): # Linear activation function
        return(s)
    
    def linearPrime(self,x): # derivative of linear activation function
        return(np.ones(len(x)))
    
    def meanSquareError(self,y, y_calculated):
        return((y_calculated - y)**2)
    
    def meanSquareErrorPrime(self,y, y_calculated):
        return(2*(y_calculated - y))
    
    def crossEntropyLoss(self,y, y_calculated):
        loss = y*np.log(y_calculated)
        return(-1*sum(loss))
    
    def forward(self, x): 
        # function of forward pass which will receive input and give the output of final layer
        # Write the forward pass using the weights to find the predicted value and return it.
        inputs = x
        for i in range(len(self.w)-1):
            z = inputs.dot(self.w[i])
            activated = self.activationHidden(z)
            inputs = activated
        z = inputs.dot(self.w[-1])
        y_calculated = self.activationOutput(z)
        return y_calculated
    
    def backward(self, x, y): 
        # find the loss and return derivative of loss w.r.t every parameter
        # Write the backpropagation algorithm here to find the gradients and return it.
        # Stores the z for  all the layers
        z_products = []
        # Stores the inputs for all the layers
        activations = [x]
        inputs = x
        for i in range(len(self.w)-1):
            z = inputs.dot(self.w[i])
            z_products.append(z)
            a = self.activationHidden(z)
            activations.append(a)
            inputs = a
        z = inputs.dot(self.w[-1])
        z_products.append(z)
        y_calculated = self.activationOutput(z)

        z_products = np.array(z_products)

        activations = np.array(activations)
        
        g_L = self.meanSquareErrorPrime(y, y_calculated)
        gl = g_L
        dw = [0]*(self.layers-1)
        dE_dwL = np.outer(activations[-1],(g_L)*(self.activationOutputPrime(z_products[-1])))
        dw[-1] = dE_dwL
        
        l = self.layers-2
        while(l>0):
            phi_prime_zl = self.activationHiddenPrime(z_products[l])
            phi_prime_zl_1 = self.activationHiddenPrime(z_products[l-1])
            #diag_phi_prime_zl = np.identity(len(phi_prime_zl)) * np.outer(np.ones(len(phi_prime_zl)), phi_prime_zl)
            gl = np.matmul(self.w[l],(phi_prime_zl*gl))
            dE_dzl = gl*(phi_prime_zl_1)
            dE_dwl = np.outer(activations[l-1],dE_dzl)
            dw[l-1] = dE_dwl # prepending the derivative
            l -= 1
        return dw
        
    def update_parameters(self, grads, learning_rate):
        # update the parameters using the gradients
        # update each parameter using the gradients and the learning rate
        for i in range(len(self.w)):
            self.w[i] -= learning_rate*grads[i]
                     
    def train(self, X, Y):     # receive the full training data set
        lr = 1e-3              # learning rate
        epochs = 50            # number of epochs
        mini_batch_size = 50   # the size of mini batch
        n = len(X)
        for e in range(epochs):
            loss = []
            mapIndexPosition = list(zip(X, Y))
            random.shuffle(mapIndexPosition)
            mini_batches = [mapIndexPosition[k:k+mini_batch_size] for k in range(0,n,mini_batch_size)]
            for mini_batch in mini_batches: 
                gradients = np.zeros_like(self.w)
                for x,y in mini_batch:
                    out = self.forward(x) # call of forward pass to get the predicted value
                    los = self.meanSquareError(y, out)
                    loss.append(sum(los))
                    grads = self.backward(x, y) # find the gradients using backward pass
                    gradients += np.array(grads)
                self.update_parameters(gradients/mini_batch_size, lr)
            print(e, sum(loss))
#             plt.plot(e,sum(loss))
#             plt.show()
        
    def predict(self,x):
        print ("Input : \n" + str(x))
        print ("Output: \n" + str((self.forward(x))))


In [ ]:
#D_in is input dimension
# H1 is dimension of first hidden layer 
# H2 is dimension of first hidden layer
#D_out is output dimension.
D_in, H1, H2, H3, D_out = 784, 300, 500, 300, 4

neurons = [D_in, H1, H2, H3, D_out] # list of number of neurons in the layers sequentially.

Hidden_activation = "sigmoid"   # activation function of the hidden layers.
Output_activation = "linear"  # activation function of the output layer.
test = Neural_Network(neurons, Hidden_activation, Output_activation )

In [ ]:
test.train(X,Y)

0 18383.868160920807


# Prediction for a data point after the training

In [8]:
test.predict(np.array([3,3]))

Input : 
[3 3]


ValueError: shapes (2,) and (784,300) not aligned: 2 (dim 0) != 784 (dim 0)

In [ ]:
i = 135
print(Y_temp[i])
plt.imshow(X[i].reshape(28,28))
plt.show()

In [ ]:
X[0].reshape(28,28).shape